# **Chapter 10: Input, Forms, and Validation**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement TextField with TextEditingController for programmatic text control
- Manage focus states using FocusNode and FocusScope for proper keyboard navigation
- Style input fields comprehensively using InputDecoration
- Build validated forms using Form widget and GlobalKey<FormState>
- Create custom validators for TextFormField with error handling
- Configure keyboard types and input formatters for specialized input
- Implement Date/Time pickers and dropdown selection widgets
- Build autocomplete and search functionality with type-ahead

---

## **Prerequisites**

- Completed Chapter 9: Material Design & Cupertino Widgets
- Understanding of StatefulWidget and state management
- Familiarity with controllers and dispose patterns
- Basic understanding of regular expressions for validation

---

## **10.1 TextField and TextEditingController**

The TextField widget allows users to enter text. TextEditingController provides programmatic control over the text, enabling you to read, write, and listen to changes.

### **Basic TextField Usage**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(TextFieldDemo());
}

class TextFieldDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: Scaffold(
        appBar: AppBar(title: Text('TextField Basics')),
        body: TextFieldExamples(),
      ),
    );
  }
}

class TextFieldExamples extends StatefulWidget {
  @override
  _TextFieldExamplesState createState() => _TextFieldExamplesState();
}

class _TextFieldExamplesState extends State<TextFieldExamples> {
  // Controllers for managing text programmatically
  late TextEditingController _nameController;
  late TextEditingController _emailController;
  late TextEditingController _passwordController;
  
  // State for showing/hiding password
  bool _obscurePassword = true;
  
  // State for character count
  int _characterCount = 0;

  @override
  void initState() {
    super.initState();
    // Initialize controllers
    _nameController = TextEditingController();
    _emailController = TextEditingController();
    _passwordController = TextEditingController();
    
    // Listen to text changes
    _nameController.addListener(_updateCharacterCount);
  }

  @override
  void dispose() {
    // Always dispose controllers to prevent memory leaks
    _nameController.removeListener(_updateCharacterCount);
    _nameController.dispose();
    _emailController.dispose();
    _passwordController.dispose();
    super.dispose();
  }

  void _updateCharacterCount() {
    setState(() {
      _characterCount = _nameController.text.length;
    });
  }

  void _clearAllFields() {
    _nameController.clear();
    _emailController.clear();
    _passwordController.clear();
    setState(() {
      _characterCount = 0;
    });
  }

  void _setTextProgrammatically() {
    _nameController.text = 'John Doe';
    // Or use text = to replace, or value = to set with selection
    _emailController.value = TextEditingValue(
      text: 'john@example.com',
      selection: TextSelection.collapsed(offset: 'john@example.com'.length),
    );
  }

  @override
  Widget build(BuildContext context) {
    return SingleChildScrollView(
      padding: EdgeInsets.all(16),
      child: Column(
        crossAxisAlignment: CrossAxisAlignment.start,
        children: [
          Text(
            'Basic TextField',
            style: Theme.of(context).textTheme.titleLarge,
          ),
          SizedBox(height: 8),
          
          // Simple TextField with controller
          TextField(
            controller: _nameController,
            decoration: InputDecoration(
              labelText: 'Full Name',
              hintText: 'Enter your full name',
              helperText: '$_characterCount characters',
              border: OutlineInputBorder(),
              prefixIcon: Icon(Icons.person),
            ),
            textCapitalization: TextCapitalization.words,
            textInputAction: TextInputAction.next,
          ),
          
          SizedBox(height: 20),
          
          // Email with specific keyboard
          TextField(
            controller: _emailController,
            decoration: InputDecoration(
              labelText: 'Email',
              hintText: 'Enter your email',
              border: OutlineInputBorder(),
              prefixIcon: Icon(Icons.email),
            ),
            keyboardType: TextInputType.emailAddress,
            textInputAction: TextInputAction.next,
          ),
          
          SizedBox(height: 20),
          
          // Password field
          TextField(
            controller: _passwordController,
            decoration: InputDecoration(
              labelText: 'Password',
              hintText: 'Enter your password',
              border: OutlineInputBorder(),
              prefixIcon: Icon(Icons.lock),
              suffixIcon: IconButton(
                icon: Icon(
                  _obscurePassword ? Icons.visibility : Icons.visibility_off,
                ),
                onPressed: () {
                  setState(() {
                    _obscurePassword = !_obscurePassword;
                  });
                },
              ),
            ),
            obscureText: _obscurePassword,
            textInputAction: TextInputAction.done,
          ),
          
          SizedBox(height: 20),
          
          // Programmatic control buttons
          Row(
            children: [
              Expanded(
                child: ElevatedButton(
                  onPressed: _setTextProgrammatically,
                  child: Text('Set Text'),
                ),
              ),
              SizedBox(width: 8),
              Expanded(
                child: ElevatedButton(
                  onPressed: _clearAllFields,
                  style: ElevatedButton.styleFrom(
                    backgroundColor: Colors.red,
                  ),
                  child: Text('Clear All'),
                ),
              ),
            ],
          ),
          
          SizedBox(height: 20),
          
          // Display current values
          Card(
            child: Padding(
              padding: EdgeInsets.all(16),
              child: Column(
                crossAxisAlignment: CrossAxisAlignment.start,
                children: [
                  Text(
                    'Current Values:',
                    style: Theme.of(context).textTheme.titleMedium,
                  ),
                  SizedBox(height: 8),
                  Text('Name: ${_nameController.text}'),
                  Text('Email: ${_emailController.text}'),
                  Text('Password: ${_passwordController.text.isEmpty ? 'Empty' : '••••••'}'),
                ],
              ),
            ),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`TextEditingController`**: Controls the text being edited. Provides:
  - `text`: Get or set the current text
  - `value`: Get or set text with selection/cursor position
  - `clear()`: Remove all text
  - `addListener()`: Listen to changes

- **`TextEditingValue`**: Contains text, selection, and composing region. Use to set cursor position programmatically.

- **`dispose()`**: Always dispose controllers in `dispose()` to prevent memory leaks.

- **`obscureText`**: Hides text (for passwords). Combine with visibility toggle button.

- **`textCapitalization`**: Controls auto-capitalization (none, sentences, words, characters).

- **`textInputAction`**: Controls the action button on keyboard (next, done, search, etc.).

- **`keyboardType`**: Shows appropriate keyboard (text, number, email, url, phone, etc.).

### **Form Validation**

```dart
import 'package:flutter/material.dart';

void main() {
  runapp(FormValidationDemo());
}

class FormValidationDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: Scaffold(
        appBar: AppBar(title: Text('Form Validation')),
        body: RegistrationForm(),
      ),
    );
  }
}

class RegistrationForm extends StatefulWidget {
  @override
  _RegistrationFormState createState() => _RegistrationFormState();
}

class _RegistrationFormState extends State<RegistrationForm> {
  // Form key for validation
  final _formKey = GlobalKey<FormState>();
  
  // Controllers
  final _nameController = TextEditingController();
  final _emailController = TextEditingController();
  final _passwordController = TextEditingController();
  final _confirmPasswordController = TextEditingController();
  final _phoneController = TextEditingController();
  final _ageController = TextEditingController();
  
  // State
  bool _obscurePassword = true;
  bool _obscureConfirm = true;
  bool _agreedToTerms = false;
  String? _selectedCountry;
  
  final List<String> _countries = [
    'United States',
    'United Kingdom',
    'Canada',
    'Australia',
    'Germany',
    'France',
    'Japan',
  ];

  @override
  void dispose() {
    _nameController.dispose();
    _emailController.dispose();
    _passwordController.dispose();
    _confirmPasswordController.dispose();
    _phoneController.dispose();
    _ageController.dispose();
    super.dispose();
  }

  void _submitForm() {
    if (_formKey.currentState!.validate()) {
      // Form is valid, process data
      _formKey.currentState!.save();
      
      // Show success
      ScaffoldMessenger.of(context).showSnackBar(
        SnackBar(
          content: Text('Registration successful!'),
          backgroundColor: Colors.green,
        ),
      );
    } else {
      // Form has errors
      ScaffoldMessenger.of(context).showSnackBar(
        SnackBar(
          content: Text('Please fix the errors'),
          backgroundColor: Colors.red,
        ),
      );
    }
  }

  @override
  Widget build(BuildContext context) {
    return SingleChildScrollView(
      padding: EdgeInsets.all(16),
      child: Form(
        key: _formKey,
        autovalidateMode: AutovalidateMode.onUserInteraction,
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              'Registration',
              style: Theme.of(context).textTheme.headlineMedium,
            ),
            SizedBox(height: 24),
            
            // Name field
            TextFormField(
              controller: _nameController,
              decoration: InputDecoration(
                labelText: 'Full Name',
                hintText: 'Enter your full name',
                prefixIcon: Icon(Icons.person),
                border: OutlineInputBorder(),
              ),
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Name is required';
                }
                if (value.length < 2) {
                  return 'Name must be at least 2 characters';
                }
                if (!RegExp(r'^[a-zA-Z\s]+$').hasMatch(value)) {
                  return 'Name can only contain letters';
                }
                return null;
              },
              textCapitalization: TextCapitalization.words,
              textInputAction: TextInputAction.next,
            ),
            
            SizedBox(height: 16),
            
            // Email field
            TextFormField(
              controller: _emailController,
              decoration: InputDecoration(
                labelText: 'Email',
                hintText: 'Enter your email',
                prefixIcon: Icon(Icons.email),
                border: OutlineInputBorder(),
              ),
              keyboardType: TextInputType.emailAddress,
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Email is required';
                }
                // Email regex pattern
                final emailRegex = RegExp(
                  r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',
                );
                if (!emailRegex.hasMatch(value)) {
                  return 'Enter a valid email';
                }
                return null;
              },
              textInputAction: TextInputAction.next,
            ),
            
            SizedBox(height: 16),
            
            // Phone field
            TextFormField(
              controller: _phoneController,
              decoration: InputDecoration(
                labelText: 'Phone Number',
                hintText: 'Enter your phone',
                prefixIcon: Icon(Icons.phone),
                border: OutlineInputBorder(),
                prefixText: '+1 ', // Country code
              ),
              keyboardType: TextInputType.phone,
              inputFormatters: [
                // Only allow digits
                // FilteringTextInputFormatter.digitsOnly,
              ],
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Phone is required';
                }
                if (value.length < 10) {
                  return 'Enter a valid phone number';
                }
                return null;
              },
              textInputAction: TextInputAction.next,
            ),
            
            SizedBox(height: 16),
            
            // Age field
            TextFormField(
              controller: _ageController,
              decoration: InputDecoration(
                labelText: 'Age',
                hintText: 'Enter your age',
                prefixIcon: Icon(Icons.cake),
                border: OutlineInputBorder(),
              ),
              keyboardType: TextInputType.number,
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Age is required';
                }
                final age = int.tryParse(value);
                if (age == null) {
                  return 'Enter a valid number';
                }
                if (age < 13) {
                  return 'Must be at least 13 years old';
                }
                if (age > 120) {
                  return 'Enter a valid age';
                }
                return null;
              },
              textInputAction: TextInputAction.next,
            ),
            
            SizedBox(height: 16),
            
            // Password field
            TextFormField(
              controller: _passwordController,
              decoration: InputDecoration(
                labelText: 'Password',
                hintText: 'Create a password',
                prefixIcon: Icon(Icons.lock),
                suffixIcon: IconButton(
                  icon: Icon(
                    _obscurePassword ? Icons.visibility : Icons.visibility_off,
                  ),
                  onPressed: () {
                    setState(() {
                      _obscurePassword = !_obscurePassword;
                    });
                  },
                ),
                border: OutlineInputBorder(),
              ),
              obscureText: _obscurePassword,
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Password is required';
                }
                if (value.length < 8) {
                  return 'Password must be at least 8 characters';
                }
                if (!RegExp(r'[A-Z]').hasMatch(value)) {
                  return 'Include at least one uppercase letter';
                }
                if (!RegExp(r'[0-9]').hasMatch(value)) {
                  return 'Include at least one number';
                }
                return null;
              },
              textInputAction: TextInputAction.next,
            ),
            
            SizedBox(height: 16),
            
            // Confirm Password
            TextFormField(
              controller: _confirmPasswordController,
              decoration: InputDecoration(
                labelText: 'Confirm Password',
                hintText: 'Re-enter your password',
                prefixIcon: Icon(Icons.lock_outline),
                suffixIcon: IconButton(
                  icon: Icon(
                    _obscureConfirm ? Icons.visibility : Icons.visibility_off,
                  ),
                  onPressed: () {
                    setState(() {
                      _obscureConfirm = !_obscureConfirm;
                    });
                  },
                ),
                border: OutlineInputBorder(),
              ),
              obscureText: _obscureConfirm,
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Please confirm your password';
                }
                if (value != _passwordController.text) {
                  return 'Passwords do not match';
                }
                return null;
              },
              textInputAction: TextInputAction.done,
            ),
            
            SizedBox(height: 16),
            
            // Country Dropdown
            DropdownButtonFormField<String>(
              value: _selectedCountry,
              decoration: InputDecoration(
                labelText: 'Country',
                prefixIcon: Icon(Icons.public),
                border: OutlineInputBorder(),
              ),
              items: _countries.map((country) {
                return DropdownMenuItem(
                  value: country,
                  child: Text(country),
                );
              }).toList(),
              onChanged: (value) {
                setState(() {
                  _selectedCountry = value;
                });
              },
              validator: (value) {
                if (value == null || value.isEmpty) {
                  return 'Please select a country';
                }
                return null;
              },
            ),
            
            SizedBox(height: 16),
            
            // Terms Checkbox
            FormField<bool>(
              initialValue: _agreedToTerms,
              validator: (value) {
                if (value != true) {
                  return 'You must agree to the terms';
                }
                return null;
              },
              builder: (FormFieldState<bool> state) {
                return Column(
                  crossAxisAlignment: CrossAxisAlignment.start,
                  children: [
                    CheckboxListTile(
                      title: Text('I agree to the Terms and Conditions'),
                      value: state.value,
                      onChanged: (value) {
                        state.didChange(value);
                        setState(() {
                          _agreedToTerms = value ?? false;
                        });
                      },
                    ),
                    if (state.hasError)
                      Padding(
                        padding: EdgeInsets.only(left: 16),
                        child: Text(
                          state.errorText!,
                          style: TextStyle(color: Colors.red, fontSize: 12),
                        ),
                      ),
                  ],
                );
              },
            ),
            
            SizedBox(height: 24),
            
            // Submit Button
            SizedBox(
              width: double.infinity,
              height: 50,
              child: ElevatedButton(
                onPressed: _submitForm,
                style: ElevatedButton.styleFrom(
                  shape: RoundedRectangleBorder(
                    borderRadius: BorderRadius.circular(8),
                  ),
                ),
                child: Text(
                  'REGISTER',
                  style: TextStyle(fontSize: 16, fontWeight: FontWeight.bold),
                ),
              ),
            ),
            
            SizedBox(height: 32),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`Form`**: A container for form fields that enables collective validation. Requires a `GlobalKey<FormState>` to access validation methods.

- **`GlobalKey<FormState>`**: Allows accessing the Form's state to call `validate()`, `save()`, and `reset()`.

- **`TextFormField`**: A TextField integrated with Form validation. Includes `validator` callback that returns error string or null.

- **`validator`**: Function that receives the current value and returns:
  - `String?`: Error message if invalid
  - `null`: If valid

- **`autovalidateMode`**: When to validate:
  - `disabled`: Only on explicit validate() call
  - `onUserInteraction`: After user interaction
  - `always`: Always validate

- **`FormField<T>`**: For non-text form controls (checkboxes, custom widgets). Manages validation state manually.

- **Validation patterns**:
  - Required: `if (value == null || value.isEmpty) return 'Required'`
  - Length: `if (value.length < 8) return 'Too short'`
  - Regex: `if (!RegExp(r'...').hasMatch(value)) return 'Invalid format'`
  - Match: `if (value != otherController.text) return 'Does not match'`

---

## **10.3 Input Formatters and Keyboard Types**

Control what users can type and which keyboard appears.

```dart
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

class InputFormattersDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Input Formatters')),
      body: SingleChildScrollView(
        padding: EdgeInsets.all(16),
        child: Column(
          children: [
            // Numbers only
            TextField(
              decoration: InputDecoration(
                labelText: 'Age (Numbers Only)',
                hintText: 'Enter your age',
              ),
              keyboardType: TextInputType.number,
              inputFormatters: [
                FilteringTextInputFormatter.digitsOnly,
              ],
            ),
            
            SizedBox(height: 16),
            
            // Decimal numbers
            TextField(
              decoration: InputDecoration(
                labelText: 'Price (Decimal)',
                hintText: '0.00',
              ),
              keyboardType: TextInputType.numberWithOptions(decimal: true),
              inputFormatters: [
                FilteringTextInputFormatter.allow(RegExp(r'^\d*\.?\d{0,2}')),
              ],
            ),
            
            SizedBox(height: 16),
            
            // Phone number with formatter
            TextField(
              decoration: InputDecoration(
                labelText: 'Phone Number',
                hintText: '(123) 456-7890',
              ),
              keyboardType: TextInputType.phone,
              inputFormatters: [
                PhoneNumberFormatter(),
              ],
            ),
            
            SizedBox(height: 16),
            
            // Credit card with formatter
            TextField(
              decoration: InputDecoration(
                labelText: 'Credit Card',
                hintText: '0000 0000 0000 0000',
              ),
              keyboardType: TextInputType.number,
              inputFormatters: [
                CreditCardFormatter(),
                LengthLimitingTextInputFormatter(19), // 16 digits + 3 spaces
              ],
            ),
            
            SizedBox(height: 16),
            
            // Uppercase text
            TextField(
              decoration: InputDecoration(
                labelText: 'Promo Code (Uppercase)',
                hintText: 'ABC123',
              ),
              textCapitalization: TextCapitalization.characters,
              inputFormatters: [
                UpperCaseTextFormatter(),
              ],
            ),
            
            SizedBox(height: 16),
            
            // Custom length limit
            TextField(
              decoration: InputDecoration(
                labelText: 'Username (max 20 chars)',
                counterText: '',
              ),
              inputFormatters: [
                LengthLimitingTextInputFormatter(20),
                // No spaces allowed
                FilteringTextInputFormatter.deny(RegExp(r'\s')),
              ],
            ),
          ],
        ),
      ),
    );
  }
}

// Custom phone number formatter
class PhoneNumberFormatter extends TextInputFormatter {
  @override
  TextEditingValue formatEditUpdate(
    TextEditingValue oldValue,
    TextEditingValue newValue,
  ) {
    var text = newValue.text;
    
    // Remove all non-digits
    text = text.replaceAll(RegExp(r'\D'), '');
    
    // Format as (XXX) XXX-XXXX
    if (text.length > 6) {
      text = '(${text.substring(0, 3)}) ${text.substring(3, 6)}-${text.substring(6)}';
    } else if (text.length > 3) {
      text = '(${text.substring(0, 3)}) ${text.substring(3)}';
    } else if (text.isNotEmpty) {
      text = '($text';
    }
    
    return TextEditingValue(
      text: text,
      selection: TextSelection.collapsed(offset: text.length),
    );
  }
}

// Custom credit card formatter
class CreditCardFormatter extends TextInputFormatter {
  @override
  TextEditingValue formatEditUpdate(
    TextEditingValue oldValue,
    TextEditingValue newValue,
  ) {
    var text = newValue.text.replaceAll(' ', '');
    
    // Add space every 4 digits
    var buffer = StringBuffer();
    for (int i = 0; i < text.length; i++) {
      if (i > 0 && i % 4 == 0) {
        buffer.write(' ');
      }
      buffer.write(text[i]);
    }
    
    var formatted = buffer.toString();
    return TextEditingValue(
      text: formatted,
      selection: TextSelection.collapsed(offset: formatted.length),
    );
  }
}

// Uppercase formatter
class UpperCaseTextFormatter extends TextInputFormatter {
  @override
  TextEditingValue formatEditUpdate(
    TextEditingValue oldValue,
    TextEditingValue newValue,
  ) {
    return TextEditingValue(
      text: newValue.text.toUpperCase(),
      selection: newValue.selection,
    );
  }
}
```

**Explanation:**

- **`TextInputFormatter`**: Base class for formatters that modify text as user types.

- **`FilteringTextInputFormatter`**:
  - `digitsOnly`: Only numbers 0-9
  - `allow(RegExp)`: Allow matching pattern
  - `deny(RegExp)`: Block matching pattern
  - `singleLineFormatter`: Prevent newlines

- **`LengthLimitingTextInputFormatter`**: Prevents input beyond max length.

- **Custom formatters**: Extend `TextInputFormatter` and implement `formatEditUpdate()`:
  - `oldValue`: Previous state
  - `newValue`: Proposed new state
  - Return modified `TextEditingValue`

- **Formatting patterns**:
  - Phone: (XXX) XXX-XXXX
  - Credit card: XXXX XXXX XXXX XXXX
  - Currency: $X,XXX.XX

- **Keyboard types**:
  - `text`: Default
  - `number`: Numeric keyboard
  - `numberWithOptions(decimal: true)`: With decimal point
  - `emailAddress`: With @ and .
  - `url`: With .com, /
  - `phone`: Phone keypad
  - `multiline`: With return key

---

## **10.4 Date/Time Pickers and Dropdowns**

Specialized input widgets for selecting dates, times, and from predefined lists.

```dart
import 'package:flutter/material.dart';
import 'package:intl/intl.dart'; // Add to pubspec.yaml

class PickersDemo extends StatefulWidget {
  @override
  _PickersDemoState createState() => _PickersDemoState();
}

class _PickersDemoState extends State<PickersDemo> {
  DateTime? _selectedDate;
  TimeOfDay? _selectedTime;
  DateTimeRange? _selectedDateRange;
  String? _selectedItem;
  String? _selectedCountry;

  final List<String> _items = ['Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5'];
  final List<Map<String, dynamic>> _countries = [
    {'name': 'USA', 'code': 'US', 'flag': '🇺🇸'},
    {'name': 'UK', 'code': 'GB', 'flag': '🇬🇧'},
    {'name': 'Canada', 'code': 'CA', 'flag': '🇨🇦'},
    {'name': 'Germany', 'code': 'DE', 'flag': '🇩🇪'},
    {'name': 'France', 'code': 'FR', 'flag': '🇫🇷'},
  ];

  Future<void> _selectDate(BuildContext context) async {
    final DateTime? picked = await showDatePicker(
      context: context,
      initialDate: _selectedDate ?? DateTime.now(),
      firstDate: DateTime(1900),
      lastDate: DateTime(2100),
      builder: (context, child) {
        return Theme(
          data: Theme.of(context).copyWith(
            colorScheme: Theme.of(context).colorScheme.copyWith(
              primary: Colors.blue,
              onPrimary: Colors.white,
              surface: Colors.white,
              onSurface: Colors.black,
            ),
          ),
          child: child!,
        );
      },
    );
    
    if (picked != null && picked != _selectedDate) {
      setState(() {
        _selectedDate = picked;
      });
    }
  }

  Future<void> _selectTime(BuildContext context) async {
    final TimeOfDay? picked = await showTimePicker(
      context: context,
      initialTime: _selectedTime ?? TimeOfDay.now(),
      builder: (context, child) {
        return MediaQuery(
          data: MediaQuery.of(context).copyWith(
            alwaysUse24HourFormat: false,
          ),
          child: child!,
        );
      },
    );
    
    if (picked != null && picked != _selectedTime) {
      setState(() {
        _selectedTime = picked;
      });
    }
  }

  Future<void> _selectDateRange(BuildContext context) async {
    final DateTimeRange? picked = await showDateRangePicker(
      context: context,
      firstDate: DateTime(2020),
      lastDate: DateTime(2025),
      initialDateRange: _selectedDateRange,
    );
    
    if (picked != null) {
      setState(() {
        _selectedDateRange = picked;
      });
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Pickers & Dropdowns')),
      body: SingleChildScrollView(
        padding: EdgeInsets.all(16),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            // Date Picker
            ListTile(
              title: Text('Date'),
              subtitle: Text(_selectedDate != null
                  ? DateFormat('MMMM dd, yyyy').format(_selectedDate!)
                  : 'Select a date'),
              trailing: Icon(Icons.calendar_today),
              onTap: () => _selectDate(context),
            ),
            
            Divider(),
            
            // Time Picker
            ListTile(
              title: Text('Time'),
              subtitle: Text(_selectedTime != null
                  ? _selectedTime!.format(context)
                  : 'Select a time'),
              trailing: Icon(Icons.access_time),
              onTap: () => _selectTime(context),
            ),
            
            Divider(),
            
            // Date Range Picker
            ListTile(
              title: Text('Date Range'),
              subtitle: Text(_selectedDateRange != null
                  ? '${DateFormat('MMM dd').format(_selectedDateRange!.start)} - '
                      '${DateFormat('MMM dd').format(_selectedDateRange!.end)}'
                  : 'Select date range'),
              trailing: Icon(Icons.date_range),
              onTap: () => _selectDateRange(context),
            ),
            
            Divider(),
            
            // Simple Dropdown
            DropdownButtonFormField<String>(
              value: _selectedItem,
              decoration: InputDecoration(
                labelText: 'Select Item',
                border: OutlineInputBorder(),
              ),
              items: _items.map((item) {
                return DropdownMenuItem(
                  value: item,
                  child: Text(item),
                );
              }).toList(),
              onChanged: (value) {
                setState(() {
                  _selectedItem = value;
                });
              },
            ),
            
            SizedBox(height: 16),
            
            // Custom Dropdown with icons
            DropdownButtonFormField<String>(
              value: _selectedCountry,
              decoration: InputDecoration(
                labelText: 'Select Country',
                border: OutlineInputBorder(),
                prefixIcon: Icon(Icons.flag),
              ),
              items: _countries.map((country) {
                return DropdownMenuItem(
                  value: country['name'] as String,
                  child: Row(
                    children: [
                      Text(country['flag'] as String),
                      SizedBox(width: 8),
                      Text(country['name'] as String),
                      Spacer(),
                      Text(
                        country['code'] as String,
                        style: TextStyle(color: Colors.grey),
                      ),
                    ],
                  ),
                );
              }).toList(),
              onChanged: (value) {
                setState(() {
                  _selectedCountry = value;
                });
              },
            ),
            
            SizedBox(height: 16),
            
            // Searchable Dropdown (Autocomplete)
            Autocomplete<String>(
              optionsBuilder: (TextEditingValue textEditingValue) {
                if (textEditingValue.text.isEmpty) {
                  return const Iterable<String>.empty();
                }
                return _countries
                    .map((c) => c['name'] as String)
                    .where((option) {
                  return option.toLowerCase().contains(
                    textEditingValue.text.toLowerCase(),
                  );
                });
              },
              onSelected: (String selection) {
                debugPrint('Selected: $selection');
              },
              fieldViewBuilder: (
                context,
                textEditingController,
                focusNode,
                onFieldSubmitted,
              ) {
                return TextFormField(
                  controller: textEditingController,
                  focusNode: focusNode,
                  decoration: InputDecoration(
                    labelText: 'Search Country',
                    hintText: 'Type to search...',
                    prefixIcon: Icon(Icons.search),
                    border: OutlineInputBorder(),
                  ),
                );
              },
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`showDatePicker`**: Material date picker. Returns `Future<DateTime?>`. Use `builder` to customize theme.

- **`showTimePicker`**: Material time picker. Returns `Future<TimeOfDay?>`. Use `MediaQuery` to control 24-hour format.

- **`showDateRangePicker`**: Select start and end dates. Returns `Future<DateTimeRange?>`.

- **`DropdownButtonFormField`**: Form-compatible dropdown with validation support. Use `items` for options.

- **`Autocomplete`**: Type-ahead search with suggestions. Provides `optionsBuilder` for filtering and `onSelected` callback.

- **`DateFormat`**: From intl package. Format dates for display.

- **Cupertino alternatives**: Use `showCupertinoModalPopup` with `CupertinoDatePicker` for iOS-style pickers.

---

## **Chapter Summary**

In this chapter, we covered input handling and form validation:

### **Key Takeaways:**

1. **TextField**: Basic text input with controller for programmatic access.

2. **TextEditingController**: 
   - Access and modify text
   - Listen to changes
   - Set selection/cursor position
   - Must be disposed

3. **FocusNode**: 
   - Control focus programmatically
   - Listen to focus changes
   - Control keyboard visibility

4. **InputDecoration**: 
   - Labels, hints, helper text
   - Icons and suffix widgets
   - Borders and styling
   - Error display

5. **Form Validation**:
   - Form widget with GlobalKey
   - TextFormField with validator
   - AutovalidateMode
   - Custom FormField for non-text inputs

6. **Input Formatters**:
   - FilteringTextInputFormatter
   - LengthLimitingTextInputFormatter
   - Custom formatters for patterns (phone, credit card)

7. **Pickers**:
   - DatePicker, TimePicker, DateRangePicker
   - Dropdowns with validation
   - Autocomplete for search

8. **Keyboard Types**: text, number, email, url, phone, multiline, etc.

### **Best Practices:**

- Always dispose controllers and focus nodes
- Use Form with validation for complex inputs
- Provide clear error messages
- Use appropriate keyboard types
- Format input in real-time for better UX
- Validate on submit and optionally on change
- Use Autocomplete for long lists instead of dropdowns

### **Next Steps:**

Now that you understand input and forms, the next chapter will cover **Chapter 11: State Management Architecture**, including:

- Ephemeral vs App state
- setState and its limitations
- Provider pattern
- Riverpod
- BLoC pattern
- Redux and MobX

---

**End of Chapter 10**

---

# **Next Chapter: Chapter 11 - State Management Architecture**

Chapter 11 will explore various state management solutions in Flutter, from simple setState to advanced patterns like BLoC and Riverpod, helping you choose the right approach for your app's complexity.